In [ ]:
from __future__ import print_function
import matplotlib.pyplot as plt
%matplotlib inline
from collections import OrderedDict
from six.moves import cPickle

import os
import sys
import timeit

import scipy.io as sio
import numpy as np
import theano
import theano.tensor as T

import nnet as nn
import criteria as er
import util
import VAE
import SVAE
import update

In [ ]:
def get_acc(pred, true):
    ll = pred - true
    ll = np.array(ll)
    acc = 1 - (np.nonzero(ll)[0].shape[0])/float(ll.shape[0])
    return acc

In [ ]:
'''Load Data'''
train_file = 'timit_train_subset_1ok.npy'
valid_file = 'timit_valid_1ok.npy'
#test_file = 'test_1ok.npy'
    
train=np.load(train_file)
valid=np.load(valid_file)
#test=np.load(test_file)

    

print('Loading training set')    
train_feat, train_label = util.shared_dataset_timit(train)
print('Loading valida')
valid_feat, valid_label = util.shared_dataset_timit(valid)
#test_feat, test_label = util.shared_dataset(test) 
    
  
'''Coefficient Initial'''        
batch_size = 300
n_epochs = 5
    
n_train_batches = train_feat.get_value(borrow=True).shape[0] // batch_size
n_valid_batches = valid_feat.get_value(borrow=True).shape[0] // batch_size
#n_test_batches = test_feat.get_value(borrow=True).shape[0] // batch_size
print('number of minibatch at one epoch: train  %i, validation %i' %
    (n_train_batches, n_valid_batches))

In [ ]:
train_label.get_value(borrow=True)

In [ ]:
z_dim = 5 #dimension of latent variable 
x_dim = train_feat.get_value(borrow=True).shape[1]
y_dim = train_label.get_value(borrow=True).shape[1]
activation = None
    
print(train_feat.get_value(borrow=True).shape[0])
print(train_label.get_value(borrow=True).shape[0])
print(train_feat.get_value(borrow=True).shape[1])
print(train_label.get_value(borrow=True).shape[1])


phi_1_struct=nn.NN_struct()
phi_1_struct.layer_dim = [x_dim+y_dim, 500, 500, z_dim]
phi_1_struct.activation = [None, None, None]
    
theta_1_struct=nn.NN_struct()
theta_1_struct.layer_dim = [x_dim, 500, 500, z_dim]
theta_1_struct.activation = [None, None, None]

theta_2_struct=nn.NN_struct()
theta_2_struct.layer_dim = [z_dim+x_dim, 500, 500, y_dim]
theta_2_struct.activation = [None, None, None]

In [ ]:
index = T.lscalar()  # index to a [mini]batch
x = T.matrix('x')  # the data is presented as rasterized images
y = T.matrix('y')  # the labels are presented as signal vector   
rng = np.random.RandomState(1234)

with open('timit_model_step_ce_2l_500h_z5.save', 'rb') as f:
    model = cPickle.load(f)

In [ ]:
model.phi_mu.HL_1.W.get_value().shape

In [ ]:
classifier_kl = SVAE.Supervised_VAE_v3_KL_2L(
    rng=rng,
    input_x = x,
    label_y = y,
    batch_size = batch_size,
    phi_1_struct = phi_1_struct,
    theta_1_struct = theta_1_struct,
    theta_2_struct = theta_2_struct,
    in_dim = x_dim,
    out_dim = y_dim,
    model = model
    )


'''
learning_rate = 0.008
cost = (classifier_kl.cost)
gparams = [T.grad(cost, param) for param in classifier_kl.params]
updates = [
    (param, param - learning_rate * gparam)
    for param, gparam in zip(classifier_kl.params, gparams)
]
'''


updates = update.adam(loss=classifier_kl.cost, all_params=classifier_kl.params, learning_rate=0.001)

#classifier_kl.phi_mu = model.phi_mu
#classifier_kl.phi_sigma = model.phi_sigma
#classifier_kl.theta_2 = model.theta_2
#classifier_kl.predict = model.predict
#print(classifier_kl.phi_sigma.OL.W.get_value())
#print(model.phi_sigma.OL.W.get_value())
#print(classifier_kl.theta_2.OL.W.get_value())
#print(model.theta_2.OL.W.get_value())

In [ ]:
print('... prepare training model')
train_model = theano.function(
    inputs=[index],
    outputs=[classifier_kl.cost, classifier_kl.predictor, classifier_kl.label_y, classifier_kl.predictor_test,
             classifier_kl.EC_mu, classifier_kl.EC_log_sigma, classifier_kl.DC_mu, classifier_kl.DC_log_sigma],
            #classifier_kl.KL, classifier_kl.CE],
    updates=updates,
    givens={
        x: train_feat[index * batch_size : (index + 1) * batch_size, :],
        y: train_label[index * batch_size : (index + 1) * batch_size, :]
    }       
)   
    
print('... prepare validate model')
validate_model = theano.function(
    inputs=[index],
    outputs=classifier_kl.cost,
    givens={
        x: valid_feat[index * batch_size : (index + 1) * batch_size, :],
        y: valid_label[index * batch_size : (index + 1) * batch_size, :]
    }        
)      

'''
print('... prepare test model')
test_model = theano.function(
    inputs=[index],
    outputs=[classifier_kl.predictor_test, classifier_kl.label_y],
    givens={
        x: test_feat[index * batch_size : (index + 1) * batch_size, :],
        y: test_label[index * batch_size : (index + 1) * batch_size, :]
    }        
)
'''



In [ ]:
n_epochs = 1000

###############
# TRAIN MODEL #
###############
'''
Define :
    xx_loss : Cost function value
    xx_score : Classification accuracy rate
'''        
    
print('... training')
    
# early-stopping parameters
patience = 10000  # look as this many examples regardless
patience_increase = 2  # wait this much longer when a new best is
                           # found
improvement_threshold = 0.995  # a relative improvement of this much is
                                   # considered significant
validation_frequency = min(n_train_batches, patience // 2)
                                  # go through this many
                                  # minibatche before checking the network
                                  # on the validation set; in this case we
                                  # check every epoch
    
#validation_frequency = n_train_batches
    
best_iter = 0
best_train_loss = np.inf
best_validation_loss = np.inf  
test_loss = np.inf
train_score = 0.
validation_score = 0.
test_score = 0.    
start_time = timeit.default_timer()

epoch = 0
done_looping = False


while (epoch < n_epochs) and (not done_looping):
    epoch = epoch + 1
    train_acc1=[]
    train_acc2=[]
    for minibatch_index in range(n_train_batches):

        [minibatch_avg_cost, pred1, lab, pred2, ec_mu, ec_log_sigma, dc_mu, dc_log_sigma] \
        = train_model(minibatch_index)
        #print(minibatch_index)
                        
        # iteration number
        iter = (epoch - 1) * n_train_batches + minibatch_index
        if iter % 100 == 0:
            print('minibatch %i/%i' % (minibatch_index, n_train_batches))
        #print(pred1)
        #print(classifier_kl.phi_sigma.OL.W.get_value())
        train_acc1.append(get_acc(pred1, np.nonzero(lab)[1]))
        train_acc2.append(get_acc(pred2, np.nonzero(lab)[1]))
            
        if (iter + 1) % validation_frequency == 0:
            # compute loss on validation set
            validation_losses = [validate_model(i) for i in range(n_valid_batches)] 
            this_validation_loss = np.mean(validation_losses)
                
            #print('CE loss: %f' % (np.mean(KL_loss)))
            #print('encoder mu: %f, sigma: %f' % (ec_m, ec_s))
            #print('decoder mu: %f, sigma: %f' % (dc_m, dc_s))
            #print(ec_m)
            #print(dc_m)
            #print(ec_s)
            #print(dc_s)
            
            #print('epoch training accuracy ec: %f, dc: %f, training loss: %f \n trainig kl: %f, ce: %f' \
            #    % (np.mean(np.array(train_acc1)), np.mean(np.array(train_acc2)),np.mean(minibatch_avg_cost), \
            #      np.mean(kl), np.mean(ce)))
            print('encoder mean: %r \n encoder log variance: %r \n decoder mean: %r \n decoder log variance: %r \n' \
                  % (ec_mu, ec_log_sigma, dc_mu, dc_log_sigma))
            print('epoch training accuracy ec: %f, dc: %f, training loss: %f' \
                % (np.mean(np.array(train_acc1)), np.mean(np.array(train_acc2)),np.mean(minibatch_avg_cost)))
            print(
                'epoch %i, minibatch %i/%i, validation loss %f' %
                (
                    epoch,
                    minibatch_index + 1,
                    n_train_batches,
                    this_validation_loss
                )
            )

            # if we got the best validation score until now
            if this_validation_loss < best_validation_loss:
                #improve patience if loss improvement is good enough
                if (
                    this_validation_loss < best_validation_loss *
                    improvement_threshold
                ):
                    patience = max(patience, iter * patience_increase)

                best_validation_loss = this_validation_loss   
                best_iter = iter
                
                '''
                # get training accuracy
                print('best training accuracy: %f' % (np.mean(np.array(train_acc))))
                # test it on the test set
                #test_losses = [test_model(i) for i in range(n_test_batches)]
                #test_score = np.mean(test_losses)

                print(('epoch %i, minibatch %i/%i, best train accuracy: %f') % \
                        (epoch, minibatch_index + 1, n_train_batches, \
                        np.mean(np.array(train_acc))))
                '''
                
                
                
                '''
                test_acc=[]
                for minibatch_index in range(n_test_batches):
                    [pred_test, lab_test]= test_model(minibatch_index)
                    #print(pred)
                    #print(np.nonzero(lab)[1])
                    # iteration number
                    iter = minibatch_index
                    test_acc.append(get_acc(pred_test, np.nonzero(lab_test)[1]))
    
                print('test accuracy: %f' % (np.mean(test_acc)))
                '''
                

        #if patience <= iter:
        #    done_looping = True
        #    break